##### 목표
 - 버섯의 특징을 활용해 독/식용 이진 분류해보자
 - Decision Tree 모델 활용해보자
 - Decision Tree 학습현황 시각화 & 과대적합 제어 (하이퍼 파라미터 튜닝)
 - 특성의 중요도를 파악 확인하기 (지니계수)

### 의사결정나무 불순도가 궁금해?
    - 지니계수가 낮으면 불순도가 낮다


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# tree 분류모델
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#### 데이터 불러오기

In [9]:
# 모든 컬럼을 표시
pd.options.display.max_columns = None

# data 변수에 담아주기
# data 폴더에 있는 mushroom 가져오기

data = pd.read_csv('./data/mushroom.csv')
data 
# stalk-root : ?데이터 들어있고, 모든 컬럼에는 문자열 데이터만 있다

poisonous cap-shape cap-surface cap-color bruises odor gill-attachment  \
0            p         x           s         n       t    p               f   
1            e         x           s         y       t    a               f   
2            e         b           s         w       t    l               f   
3            p         x           y         w       t    p               f   
4            e         x           s         g       f    n               f   
...        ...       ...         ...       ...     ...  ...             ...   
8119         e         k           s         n       f    n               a   
8120         e         x           s         n       f    n               a   
8121         e         f           s         n       f    n               a   
8122         p         k           y         n       f    y               f   
8123         e         x           s         n       f    n               a   

     gill-spacing gill-size gill-color stalk-shape stalk-root  \
0               c         n          k           e          e   
1               c         b          k           e          c   
2               c         b          n           e          c   
3               c         n          n           e          e   
4               w         b          k           t          e   
...           ...       ...        ...         ...        ...   
8119            c         b          y           e          ?   
8120            c         b          y           e          ?   
8121            c         b          n           e          ?   
8122            c         n          b           t          ?   
8123            c         b          y           e          ?   

     stalk-surface-above-ring stalk-surface-below-ring stalk-color-above-ring  \
0                           s                        s                      w   
1                           s                        s                      w   
2                           s                        s                      w   
3                           s                        s                      w   
4                           s                        s                      w   
...                       ...                      ...                    ...   
8119                        s                        s                      o   
8120                        s                        s                      o   
8121                        s                        s                      o   
8122                        s                        k                      w   
8123                        s                        s                      o   

     stalk-color-below-ring veil-type veil-color ring-number ring-type  \
0                         w         p          w           o         p   
1                         w         p          w           o         p   
2                         w         p          w           o         p   
3                         w         p          w           o         p   
4                         w         p          w           o         e   
...                     ...       ...        ...         ...       ...   
8119                      o         p          o           o         p   
8120                      o         p          n           o         p   
8121                      o         p          o           o         p   
8122                      w         p          w           o         e   
8123                      o         p          o           o         p   

     spore-print-color population habitat  
0                    k          s       u  
1                    n          n       g  
2                    n          n       m  
3                    k          s       u  
4                    n          a       g  
...                ...        ...     ...  
8119                 b          c       l  
8120                 b          v       l  
8121             

### 데이터 전처리 및 탐색
 - 문제(특성)와 답(레이블) 분리
 - x,y 간단한 통계량 확인
 - 머신러닝 모델은 숫자마을 인식할 수 있음 ; 문자=>숫자
 - 훈련셋, 테스트셋 분리

In [10]:
# 답 데이터 : poisonous,
# 문제 데이터 : 나머지
X = data.loc[:, 'cap-shape':]
y = data.loc[:, 'poisonous']
data

poisonous cap-shape cap-surface cap-color bruises odor gill-attachment  \
0            p         x           s         n       t    p               f   
1            e         x           s         y       t    a               f   
2            e         b           s         w       t    l               f   
3            p         x           y         w       t    p               f   
4            e         x           s         g       f    n               f   
...        ...       ...         ...       ...     ...  ...             ...   
8119         e         k           s         n       f    n               a   
8120         e         x           s         n       f    n               a   
8121         e         f           s         n       f    n               a   
8122         p         k           y         n       f    y               f   
8123         e         x           s         n       f    n               a   

     gill-spacing gill-size gill-color stalk-shape stalk-root  \
0               c         n          k           e          e   
1               c         b          k           e          c   
2               c         b          n           e          c   
3               c         n          n           e          e   
4               w         b          k           t          e   
...           ...       ...        ...         ...        ...   
8119            c         b          y           e          ?   
8120            c         b          y           e          ?   
8121            c         b          n           e          ?   
8122            c         n          b           t          ?   
8123            c         b          y           e          ?   

     stalk-surface-above-ring stalk-surface-below-ring stalk-color-above-ring  \
0                           s                        s                      w   
1                           s                        s                      w   
2                           s                        s                      w   
3                           s                        s                      w   
4                           s                        s                      w   
...                       ...                      ...                    ...   
8119                        s                        s                      o   
8120                        s                        s                      o   
8121                        s                        s                      o   
8122                        s                        k                      w   
8123                        s                        s                      o   

     stalk-color-below-ring veil-type veil-color ring-number ring-type  \
0                         w         p          w           o         p   
1                         w         p          w           o         p   
2                         w         p          w           o         p   
3                         w         p          w           o         p   
4                         w         p          w           o         e   
...                     ...       ...        ...         ...       ...   
8119                      o         p          o           o         p   
8120                      o         p          n           o         p   
8121                      o         p          o           o         p   
8122                      w         p          w           o         e   
8123                      o         p          o           o         p   

     spore-print-color population habitat  
0                    k          s       u  
1                    n          n       g  
2                    n          n       m  
3                    k          s       u  
4                    n          a       g  
...                ...        ...     ...  
8119                 b          c       l  
8120                 b          v       l  
8121             

In [11]:
# x,y 간단한 통계량 확인
# 데이터프레임.describe()
print(y.describe())
X.describe()

count     8124
unique       2
top          e
freq      4208
Name: poisonous, dtype: object


cap-shape cap-surface cap-color bruises  odor gill-attachment  \
count       8124        8124      8124    8124  8124            8124   
unique         6           4        10       2     9               2   
top            x           y         n       f     n               f   
freq        3656        3244      2284    4748  3528            7914   

       gill-spacing gill-size gill-color stalk-shape stalk-root  \
count          8124      8124       8124        8124       8124   
unique            2         2         12           2          5   
top               c         b          b           t          b   
freq           6812      5612       1728        4608       3776   

       stalk-surface-above-ring stalk-surface-below-ring  \
count                      8124                     8124   
unique                        4                        4   
top                           s                        s   
freq                       5176                     4936   

       stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
count                    8124                   8124      8124       8124   
unique                      9                      9         1          4   
top                         w                      w         p          w   
freq                     4464                   4384      8124       7924   

       ring-number ring-type spore-print-color population habitat  
count         8124      8124              8124       8124    8124  
unique           3         5                 9          6       7  
top              o         p                 w          v       d  
freq          7488      3968              2388       4040    3148

### X.describe()
- unique : 종류 개수
- top    : 가장 빈번한 항목
- freq   : 가장 빈번한 항목의 갯수

#### X.iloc[:,1].value_counts()
- 각항목별 더 자세한 값 확인

In [12]:
X.iloc[:,1].value_counts()

y    3244
s    2556
f    2320
g       4
Name: cap-surface, dtype: int64

# 데이터 전처리
## 1. 레이블 인코딩

## 2. 원핫 인코딩

 - 값의 크고 작음이 의미가 있을 때 : 레이블 인코딩
 - 값의 크고 작음이 의미가 없을 때 : 원핫 인코딩 (컬럼 개수가 늘어 성능이 떨어질 수 있다)


In [13]:
# 원핫 인코딩
# X만 하면 된다.
X_one_hot = pd.get_dummies(X)
X= X_one_hot

In [14]:
# 훈련셋, 테스트셋 분리 (7:3) ,랜덤 스테이트 7
X_train, X_test = train_test_split(X, test_size=0.3, random_state=7)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=7)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5686, 117) (2438, 117) (5686,) (2438,)


### 모델링
 - 모델 생성
 - 모델 학습
 - 교차검증 진행해보기
 - 모델 예측 및 평가
 - 특성의 중요도 확인
 - 학습 현황 시각화 (tree-white box)
 - 과대적합을 제어하는 파라미터 연결해보기

In [15]:
# 하이퍼 파라미터를 별도 정의 X - 기본값으로 연결
# tree_model 변수에 모델 생성
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train,y_train)
pre = tree_model.predict(X_test)


# accuracy_score 없어도 모델.score(X,y)로 정확도 예측 가능
print("accuracy_score 사용 :", accuracy_score(y_test, pre))
print("모델.score 사용 :",tree_model.score(X,y))

accuracy_score 사용 : 1.0
모델.score 사용 : 1.0


In [16]:
# 교차검증
from sklearn.model_selection import cross_val_score
cv_result = cross_val_score(tree_model, X_train, y_train, cv=5)
print(cv_result, cv_result.mean())

[1.         1.         1.         0.99912049 1.        ] 0.9998240985048372


In [17]:
# feature importances 확인
# X 컬럼의 각 중요도를 확인한다
# 모든 중요도를 합치면 1이됨
fi = tree_model.feature_importances_
fi.shape

(117,)

In [18]:
X_train.columns

Index(['cap-shape_b', 'cap-shape_c', 'cap-shape_f', 'cap-shape_k',
       'cap-shape_s', 'cap-shape_x', 'cap-surface_f', 'cap-surface_g',
       'cap-surface_s', 'cap-surface_y',
       ...
       'population_s', 'population_v', 'population_y', 'habitat_d',
       'habitat_g', 'habitat_l', 'habitat_m', 'habitat_p', 'habitat_u',
       'habitat_w'],
      dtype='object', length=117)

In [19]:
# 중요도 => df 변환, 컬럼명 연결
fi_df = pd.DataFrame(fi, index=X_train.columns, columns=['feature_importances'])
fi_df.sort_values(by='feature_importances',ascending=False)

feature_importances
odor_n                          0.608902
stalk-root_c                    0.181454
stalk-root_r                    0.084697
spore-print-color_r             0.036178
odor_l                          0.025074
...                                  ...
gill-color_e                    0.000000
gill-color_b                    0.000000
gill-size_b                     0.000000
gill-spacing_w                  0.000000
habitat_w                       0.000000

[117 rows x 1 columns]